## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
print(data.info())
data.head(15)

FileNotFoundError: [Errno 2] File b'/datasets/data.csv' does not exist: b'/datasets/data.csv'

### Вывод

Есть пропуски в двух столбцах: days_employed и total_income, причем количество строк с пропущенными значениями по первому и второму столбцам одинаково.
Также при беглом осмотре заметны ошибки в виде минусов перед значениями, а в графе с образованием встречаются и строчные и заглавные буквы.
Данные автоматически записались в форматах int64 и float64, хотя, наверняка, эти форматы излишни.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
# начнем со столбца days_employed
# проверим максимальное, минимальное значения, найдем среднее и медиану
print('max days_employed:', data.days_employed.max())
print('min days_employed:', data.days_employed.min())
print('mean days_employed:', data.days_employed.mean())
print('median days_employed:', data.days_employed.median())
print()

# минимальное значение меньше 0, чего быть не может
# вероятно, минус вызван технической ошибкой, переведем отрицательные значения в положительные
data.days_employed = data.days_employed.abs()

# максимальное и среднее значения слишком велики, чтобы быть верными
# имеем дело с выбросами, причем, судя по среднему, их количество внушительно.
# у нас нет данных, чтобы исправить значения на верные, поэтому заменим их на NaN
        
# теперь взглянем на строки с пропущенными значениями:
# в тех же строках пропущено и значение с доходами
# (помним, что количество пропусков и там и там одинаково, значит это неслучайные пропуски)
# мое предположение - люди не указали свои доходы и трудовой стаж нарочно,
# однако выявить причину на основе имеющихся данных не представляется возможным
# поэтому пропуски отнесем к неслучайным (MNAR - пропускам),
# а значит заполнять их стоит внимательно

# при этом строк с пропущенными значениями слишком много, чтобы их игнорировать
# воспользуемся заполнением с пристрастным подбором (подбором внутри групп)

# известно, что есть корреляция между доходами и уровнем образования.
# разобьем людей на группы по уровню образования и посчитаем медианный уровень дохода внутри групп

# посмотрим, какие есть группы по образованию и количество человек в них (воспользуемся столбцом education_id)
print('||| education_id value_counts |||')
print(data.education_id.value_counts())
print()

# стоило бы рассчитать средний стаж внутри группы, но
# возьму медианное значение, т.к. пока не смог обработать выбросы в столбце days_employed
# print('||| median_days_employed grouped by education |||')
# print(data.groupby('education_id').days_employed.median())
# print()

# рассчитаем медианный доход внутри каждой группы
# print('||| median_total_income grouped by education |||')
# print(data.groupby('education_id').total_income.median())
# print()
unique_education_ids = len(data.education_id.unique())
print(unique_education_ids)
# создаем список со значениями медианного стажа
median_days_employed = []
for i in range(unique_education_ids):
    median_days_employed.append(data[data.education_id == i].days_employed.median())
print(median_days_employed)

# создаем список со значениями медианной зарплаты
median_total_income = []
for i in range(unique_education_ids):
    median_total_income.append(data[data.education_id == i].total_income.median())
print(median_total_income)

# заменим пропущенные значения в столбце days_employed на средние по группе с помощью функции
def fix_nan_days_employed(row):
    education_id = row.education_id
    if row.days_employed != row.days_employed:
        return median_days_employed[education_id]
    else:
        return row.days_employed

# заменим пропущенные значения в столбце total_income на средние по группе с помощью функции
def fix_nan_total_income(row):
    education_id = row.education_id
    if row.total_income != row.total_income:
        return median_total_income[education_id]
    else:
        return row.total_income
    
# применяем созданные функции            
data.days_employed = data.apply(fix_nan_days_employed, axis=1)
data.total_income = data.apply(fix_nan_total_income, axis=1)

# проверим как сработала функция, взглянув таблицу
print(data.head(15))

# окончательная проверка вызовом info()
print(data.info())

### Вывод

Пропуски в столбцах days_employed и total_income заполнены пристрастным подбором (по группам). Пропущенные значения заполнялись средними в по группам, сформированными по с уровню образования.
Замена пропусков прошла успешно, это можно увидеть на примере строки с индексом 12, которая раньше содержала пустые значения.
Судя по всему, люди преднамеренно не указали свой. стаж и доход.
Проверка всей таблицы с помощью метода info() также показала, что строк с пустыми значениями больше нет.

### Замена типа данных

In [ ]:
# еще раз взглянем на таблицу
print(data.head())
print()
print(data.info())
print()

# два столбца (days_employed и total_income) имеют тип данных float64
# для оптимального использования памяти, заменим их на целочисленный тип int64
data.days_employed = data.days_employed.astype(int)
data.total_income = data.total_income.astype(int)

#данные после замены
data.head()

### Вывод

Замену вещественных чисел на целочисленные проведено методом astype, т.к. округление в меньшую сторону не вызовет сколь-нибудь значительного искажения данных.

### Обработка дубликатов

In [ ]:
# в данных присутствуют строковые значения, они могут привести ошибкам в поисках дубликатов,
# если их не привести к одинаковому виду (все заглавные буквы привести к строчным)

# данные до изменения
print(data.head())
print()

# создадим массив с названиями столбцов, содержащими строковый тип данных
object_columns = []

# заносим в массив названия столбцов
for i in range(len(data.columns)):
    if data.dtypes[i] == object:
        object_columns.append(data.columns[i])
        
# проверяем элементы массива
print(object_columns)
print()

# приводим все столбцы со строковыми значениями к строчным буквам
for element in object_columns: 
    data[element] = data[element].str.lower()

# проверим количество дубликатов
print('duplicated before:', data.duplicated().sum())
print()

# удалим дублирующиеся строки
data = data.drop_duplicates().reset_index(drop=True)

# проверим количество дубликатов после удаления
print('duplicated after:', data.duplicated().sum())
print()

# данные после изменения
data.info()

### Вывод

Дублирующихся строк оказалось сравнительно немного. Их наличие, вероятно, вызвано техническими ошибками в системе, также некоторые пользователи могли оставлять свои данные несколько раз.
Дубликаты обнаруживаются методом duplicated(), поэтому удалены вызовом метода drop_duplicates() с последующим сбросом индекса методом reset_index()

### Лемматизация

In [54]:
# импортируем библиотеку pymystem3 для последующей лемматизации
from pymystem3 import Mystem
m = Mystem()

# создадим столбец с лемматизированными значениями столбца purpose
data['lemmas'] = data.purpose.apply(m.lemmatize)


# проверяем содержимое нового столбца
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


### Вывод

Лемматизация проведена с помощью библиотеки pymystem3, леммы из столбца 'purpose' записаны в новый столбец 'lemmas'

### Категоризация данных

In [55]:
# проведем категоризацию в соответствии с причинами взятия кредита

# для этого посмотрим на уникальные значения столбца purpose
print(data.purpose.unique())
print()

# из этих значений можно выделить следующие категории:
# 1. сделки с недвижимостью (жильем)
# 2. сделки с автомобилями
# 3. образование
# 4. свадьбы

# создадим новый список purpose_id в соответствии с причинами взятия кредита
purpose_id = []
for i in range(len(data)):
    if 'недвижимость' in data.lemmas[i] or 'жилье' in data.lemmas[i]:
        purpose_id.append(0)
    elif 'автомобиль' in data.lemmas[i]:
        purpose_id.append(1)
    elif 'образование' in data.lemmas[i]:
        purpose_id.append(2)
    elif 'свадьба' in data.lemmas[i]:
        purpose_id.append(3)

# проверим соответствие длины полученного списка и длины датафрейма
print('purpose_id len:', len(purpose_id))
print('data len:', len(data))
print()

# длины совпадают, можно добавлять столбец в таблицу
data['purpose_id'] = purpose_id
#print(data.head())


# проведем категоризацию в соответствии с уровнем зарплат
# разделим всех клиентов на 4 равные по количеству группы

# для определения граничных зарплат воспользуемся медианой
median_2_3 = data.total_income.median()
print('median between 2 and 3 group', median_2_3)
print()

median_1_2 = data[data.total_income < median_2_3].total_income.median()
print('median between 1 and 2 group', median_1_2)
print()

median_3_4 = data[data.total_income > median_2_3].total_income.median()
print('median between 3 and 4 group', median_3_4)
print()

# получили следующие границы для групп:
# ЗП < 107620 - 1 группа
# 107620 < ЗП < 141147 - 2 группа
# 141147 < ЗП < 195818 - 3 группа
# ЗП > 195818 - 4 группа

# создадим новый список income_id, в который запишем зарплатные группы сотрудников индексами соответственно от 0 до 3
income_id = []
for i in range(len(data)):
    if data.total_income[i] <= median_1_2:
        income_id.append(0)
    elif median_1_2 < data.total_income[i] <= median_2_3:
        income_id.append(1)
    elif median_2_3 < data.total_income[i] <= median_3_4:
        income_id.append(2)
    elif data.total_income[i] > median_1_2:
        income_id.append(3)
        
# проверим соответствие длины полученного списка и длины датафрейма
print('income_id len:', len(income_id))
print('data len:', len(data))
print()

# длины совпадают, можно добавлять столбец в таблицу
data['income_id'] = income_id

# проверим, что добавление прошло успешно и количество людей в каждой группе совпадает
print(data.income_id.value_counts())
data.head()

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']

purpose_id len: 21454
data l

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_id,income_id
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",0,3
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",1,1
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",0,2
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",2,3
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",3,2


### Вывод

Большая часть необходимых данных уже категоризована, необходимо было провести категоризацию по столбцу purpose,
т.е. разделить все цели кредитов на группы и присвоить идентификаторы (в этом случае от 0 до 3) в соответствии с целями:
1. сделки с недвижимостью
2. сделки с автомобилями
3. получение образования
4. проведение свадьбы

Для категоризации использовался созданный в предыдущем шаге столбец lemmas.
Идентификаторы записаны в новый столбец purpose_id.

Также для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" была проведена категоризация по ежемесячному уровню дохода. Люди были поровну поделены на 4 группы:
1. доход меньше 107620 р.
2. доход в пределах от 107620 р. до 141147 р.
3. доход в пределах от 141147 р. до 195818 р.
4. доход больше 195818 р.

Идентификаторы записаны в новый столбец income_id.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [56]:
# для ответа на вопрос стоит педварительно убедиться, что в столбце children корректные данные
print('||| Before correction |||')
print(data.children.value_counts())
print()

# видим, что присутствуют 47 отрицательных значений, чего быть не может
# также неестественно выглядят 76 значений "20" детей, т.к. нет других значений в диапазоне от 6 до 19 детей

# заменим значение 20 на -1, чтобы впоследствии учитывать только строки c неотрицательными значениями
data.loc[data.children == 20, 'children'] = -1

# проверим успешность замены
print('||| After correction |||')
print(data.children.value_counts())
print()

# теперь посмотрим, есть ли зависимость между значениями стобцов children и debt
print(data[data.children >= 0].pivot_table(index='children', columns='debt', values='gender', aggfunc='count'))
print()

# для наглядности посчитаем отношение должников в каждой группе к количеству людей в соответствующей группе

# общее количество людей в группе
total_children_data = data[data.children >=0].children.value_counts()
print('||| Total men sorted by children |||')
print(total_children_data)
print()

# количество должников в группе
debt_by_children_data = data[data.children >= 0].pivot_table(index='children', columns='debt', values='gender', aggfunc='count')[1]
print('||| Debt men sorted by children |||')
print(debt_by_children_data)
print()

# отношение должников к общему количеству
debt_precent_by_children = (debt_by_children_data / total_children_data)*100
print('||| debt % by children |||')
print(debt_precent_by_children)
print()

||| Before correction |||
 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

||| After correction |||
 0    14091
 1     4808
 2     2052
 3      330
-1      123
 4       41
 5        9
Name: children, dtype: int64

debt            0       1
children                 
0         13028.0  1063.0
1          4364.0   444.0
2          1858.0   194.0
3           303.0    27.0
4            37.0     4.0
5             9.0     NaN

||| Total men sorted by children |||
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

||| Debt men sorted by children |||
children
0    1063.0
1     444.0
2     194.0
3      27.0
4       4.0
5       NaN
Name: 1, dtype: float64

||| debt % by children |||
children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5         NaN
dtype: float64



### Вывод

Наличие детей действительно влияет на вероятность возврата кредита в срок.
В среднем, люди с детьми на 1.5% - 2% чаще становятся должниками.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
# посмотрим, какие есть группы людей по семейному положения
total_family_status_data = data.groupby('family_status_id').family_status.value_counts()
print('||| Total family status count |||')
print(total_family_status_data)
print()

# количество должников в зависимости от семейного положения
debt_by_family_status = data.groupby('family_status_id').debt.sum()
print('||| Debt by family status count |||')
print(debt_by_family_status)
print()

# отношение должников к общему количеству людей в группе
debt_precent_by_family_status = (debt_by_family_status / total_family_status_data)*100
print('||| Debt % by family status |||')
print(debt_precent_by_family_status)
print()

||| Total family status count |||
family_status_id  family_status        
0                 женат / замужем          12339
1                 гражданский брак          4151
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 не женат / не замужем     2810
Name: family_status, dtype: int64

||| Debt by family status count |||
family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64

||| Debt % by family status |||
family_status_id  family_status        
0                 женат / замужем          7.545182
1                 гражданский брак         9.347145
2                 вдовец / вдова           6.569343
3                 в разводе                7.112971
4                 не женат / не замужем    9.750890
dtype: float64



### Вывод

Семейное положение также влияет на вероятность возврата кредита в срок.
В среднем, меньше всего должников среди вдов/вдовцов и людей в разводе
Больше всего должников среди людей не состоящих в браке, а также среди заключивших гражданский брак.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [58]:
# для ответа на вопрос воспользуемся ранее созданным столбцом income_id,
# в котором указаны группы людей по целям кредита

# cводная таблица показывающая количество должников в каждой группе
print(data.pivot_table(index='income_id', columns='debt', values='total_income', aggfunc='count'))
print()

# общее количество людей в каждой группе
print('||| Total income_id count |||')
total_income_id_data = data.income_id.value_counts(sort=False)
print(total_income_id_data)
print()

# количество должников в каждой группе
print('||| Debt by income_id count |||')
debt_by_income_id = data.groupby('income_id').debt.sum()
print(debt_by_income_id)
print()

# отношение должников к общему количеству людей в группе
print('||| Debt % by income_id |||')
debt_precent_by_income_id = (debt_by_income_id / total_income_id_data)*100
print(debt_precent_by_income_id)
print()

debt          0    1
income_id           
0          4937  427
1          4890  473
2          4906  458
3          4980  383

||| Total income_id count |||
0    5364
1    5363
2    5364
3    5363
Name: income_id, dtype: int64

||| Debt by income_id count |||
income_id
0    427
1    473
2    458
3    383
Name: debt, dtype: int64

||| Debt % by income_id |||
income_id
0    7.960477
1    8.819690
2    8.538404
3    7.141525
dtype: float64



### Вывод

Согласно полученным данным, чаще всего возвращают кредит в срок люди, чей уровень дохода выше 195818р в месяц (7.14% должников)
Самый большой процент людей, просрачивающих выплаты по кредитам среди тех,
чей доход лежит в пределах от 141147р до 107620р в месяц - 8.82% должников

- Как разные цели кредита влияют на его возврат в срок?

In [59]:
# для ответа на вопрос воспользуемся ранее созданным столбцом purpose_id,
#  в котором указаны группы людей по уровню дохода

# cводная таблица показывающая количество должников в каждой группе
print(data.pivot_table(index='purpose_id', columns='debt', values='purpose', aggfunc='count'))
print()

# общее количество людей в каждой группе
print('||| Total purpose_id count |||')
total_purpose_id_data = data.purpose_id.value_counts(sort=False)
print(total_purpose_id_data)
print()

# количество должников в каждой группе
print('||| Debt by purpose_id count |||')
debt_by_purpose_id = data.groupby('purpose_id').debt.sum()
print(debt_by_purpose_id)
print()

# отношение должников к общему количеству людей в группе
print('||| Debt % by purpose_id |||')
debt_precent_by_purpose_id = (debt_by_purpose_id / total_purpose_id_data)*100
print(debt_precent_by_purpose_id)
print()

debt            0    1
purpose_id            
0           10029  782
1            3903  403
2            3643  370
3            2138  186

||| Total purpose_id count |||
0    10811
1     4306
2     4013
3     2324
Name: purpose_id, dtype: int64

||| Debt by purpose_id count |||
purpose_id
0    782
1    403
2    370
3    186
Name: debt, dtype: int64

||| Debt % by purpose_id |||
purpose_id
0    7.233373
1    9.359034
2    9.220035
3    8.003442
dtype: float64



### Вывод

Чаще всего возвращают кредит в срок люди, которые брали кредит на покупку жилья 
Больше процент людей, просрачивающих выплаты по кредитам среди людей,
которые в целях кредита указали "образование" и "приобритение автомобиля".

### Шаг 4. Общий вывод

Была проведена предобработка данных, полученны ответы на все поставленные вопросы.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.